In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [3]:
# all_files=glob.glob("*.csv")

In [4]:
data=pd.DataFrame()
for p in all_files:
    d=pd.read_csv(p,sep="\t")
    d["label"]=re.sub(r"[a-zA-Z0-9]","",p)
    data=pd.concat([data,d])

NameError: name 'all_files' is not defined

In [44]:
data.to_csv("all_data_comments.csv",index=False,sep="\t")

In [46]:
X=data["content"].values
Y=data["label"].values

In [59]:
X_train,x_test,Y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=1000)

In [85]:
len(x_test)

16143

In [50]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

In [86]:
X_train_tfidf=tfidf_vect.fit_transform(X_train)
x_test_tfidf=tfidf_vect.transform(x_test)

In [88]:
x_test_tfidf

<16143x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 77803 stored elements in Compressed Sparse Row format>

In [60]:
Ytrain=LabelEncoder().fit_transform(Y_train)

In [75]:
Ytrain[-1]

4

In [79]:
Ytest=LabelEncoder().fit_transform(y_test)

In [70]:
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, eta=0.3, silent=1, subsample=0.8) 
scores=cross_val_score(xgb_model,X_train_tfidf,Ytrain,cv=5)
scores

C:\Users\khaled.15\AppData\Roaming\Python\Python37\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:10:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but

array([0.97332095, 0.97172441, 0.97132616, 0.97570689, 0.97225541])

In [131]:
sum(scores)/len(scores)

0.9685392978090455

In [80]:
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, eta=0.3, silent=1, subsample=0.8) 
xgb_model.fit(X_train_tfidf,Ytrain)

C:\Users\khaled.15\AppData\Roaming\Python\Python37\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:18:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:18:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=80, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=4,
              reg_lambda=1, scale_pos_weight=None, silent=1, subsample=0.8,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [89]:
predictions=xgb_model.predict(x_test_tfidf)

In [90]:
print(classification_report(predictions,Ytest))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1202
           1       0.98      0.92      0.95      3058
           2       0.98      0.98      0.98      3050
           3       0.97      0.98      0.98      2992
           4       0.95      0.99      0.97      2851
           5       0.99      0.99      0.99      2990

    accuracy                           0.97     16143
   macro avg       0.97      0.98      0.97     16143
weighted avg       0.97      0.97      0.97     16143



In [92]:
len(X_train)

37666

In [93]:
16143+37666

53809

In [94]:
data["label"].value_counts()

الحمدلله عالسلامة.    10000
مبارك ومبروك.         10000
هههههه.               10000
يرحمه ويرحمو.         10000
 سعر نقطة.....         9604
  صباح ومسا.           4205
Name: label, dtype: int64

In [121]:
from sklearn.svm import SVC

In [128]:
l_model=SVC(C=50
           )

In [129]:
scores=cross_val_score(l_model,X_train_tfidf,Ytrain)

In [130]:
scores

array([0.96734802, 0.96734369, 0.96681269, 0.97159166, 0.96960042])

NotFittedError: This RidgeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.